In [31]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt

## Start date
start = '2009-06-10'
## CoinMetrics data
data = data0 = pd.read_csv('https://community-api.coinmetrics.io/v2/assets/btc/metricdata.csv?metrics=PriceUSD,CapMrktCurUSD,CapRealUSD,TxTfrValAdjUSD,FeeTotUSD,FeeTotNtv,IssContNtv,BlkCnt,DiffMean,SplyCur&start='+start, index_col = 0, parse_dates=True)

## Blockchain.com wallets data
timespan = 'all'
w_data = pd.read_csv('https://api.blockchain.info/charts/my-wallet-n-users?timespan='+timespan+'&format=csv', header=None, infer_datetime_format =True)
w_data.columns = ['Date', 'nWallets']
## Parsing date
w_data['Date'] = w_data['Date'].apply(lambda x: datetime.strptime(str(x).split(' ')[0], '%Y-%m-%d'))
## Eliminate 'intra day' data
w_data['Date'] = w_data['Date'].drop_duplicates(keep = 'last')
w_data = w_data[w_data['Date'].notnull()]
## Date to index
w_data.set_index('Date', inplace=True, drop=True)
## Merge dataframes
data = pd.concat([data, w_data], axis=1, sort=True, join='outer')
## Fill empty cells
data['nWallets'] = data['nWallets'].ffill(axis = 0)

In [2]:
## MVRV

## Realized Price
data['Realized Price'] = data['CapRealUSD'] / data['SplyCur']

## MVRV Ratio
data['MVRV'] = data['CapMrktCurUSD'] / data['CapRealUSD']
## MVRV-z
per = 200
data['MVRV-z'] = (data['MVRV'] - data['MVRV'].rolling(per).mean()) / data['MVRV'].rolling(per).std()

## Price to MVRV (PTM)
data['PTM'] = data['PriceUSD'] * data['MVRV'].mean()

In [3]:
## NVT

## NVT Ratio
data['NVT'] = data['PriceUSD'] / data['TxTfrValAdjUSD']
## NVT Price
per = 200
data['NVT Price'] = (data['NVT'].mean() * data ['TxTfrValAdjUSD']).rolling(per).mean()

In [56]:
## Avoided Costs

## Open csv
avdd_cost = pd.read_csv('Avoided Costs.csv')
avdd_cost.columns = ('Date', 'AC Price')
## Fix index
avdd_cost.set_index('Date', inplace=True)
avdd_cost.index = pd.to_datetime(avdd_cost.index)
## Join DataFrame
data = data.join(avdd_cost)

In [57]:
## Cost of Production

## Open csv
cost_prod = pd.read_csv('Data/Edwards model.csv')
## Fix index
cost_prod.set_index('Date', inplace=True)
cost_prod.index = pd.to_datetime(cost_prod.index)
## Join DataFrame
data = data.join(cost_prod)

In [58]:
## Stock-to-Flow

## S2F Ratio
data['S2F'] = data['SplyCur'] / data['IssContNtv'].rolling(360).sum()

## S2F Price
data['S2F Price'] = np.exp(-1.84) * data['S2F'] ** 3.36
## S2F 360d discounting Satoshi's tokens (1MM BTC)
data['disc S2F'] = (data['SplyCur'] - 1000000) / data['IssContNtv'].rolling(360).sum()

In [54]:
## Metcalfe's Law

## Transaction pairs (mil)
data['Tx Pairs'] = data['nWallets'] * (data['nWallets'] - 1) / 2
## Gompertz Sigmoid
data['G_Sigmoid'] = data['SplyCur'] * np.log(21000000 / data['SplyCur'])
## Metcalfe value
a = 3.5
b = 500000
data['Metcalfe (log)'] = a + b * np.log(data['Tx Pairs']) / data['G_Sigmoid']
data['Metcalfe'] = np.exp(data['Metcalfe (log)'])

In [55]:
data.to_csv('metrics.csv')